In [ ]:
import pandas as pd
import numpy as np
import json
import plotly.graph_objects as go
import plotly.express as px
from google.colab import files

print("✅ All libraries imported successfully!")

✅ All libraries imported successfully!


In [ ]:
"""
Clean the patent data and prepare it for visualization
"""
print("Loading and cleaning patent data...")

# Load the CSV with proper encoding
df = pd.read_csv('Pre-Aggregated MSA Data Cleaned.csv', encoding='latin-1')

print(f"Original shape: {df.shape}")

# Remove empty rows
df_cleaned = df.dropna(how='all')

# Remove rows where key columns are missing
df_cleaned = df_cleaned.dropna(subset=['ID Code', 'U.S. Regional Title'])

# Convert ID Code to string and pad with leading zeros to match GeoJSON
df_cleaned['ID Code'] = df_cleaned['ID Code'].astype(int).astype(str).str.zfill(5)

# Convert year columns to numeric
year_columns = ['2000', '2001', '2002', '2003', '2004', '2005', '2006',
                '2007', '2008', '2009', '2010', '2011', '2012', '2013',
                '2014', '2015']

for year in year_columns:
    df_cleaned[year] = pd.to_numeric(df_cleaned[year], errors='coerce')

# Remove any remaining rows with all NaN values in year columns
df_cleaned = df_cleaned.dropna(subset=year_columns, how='all')

print(f"✅ Cleaned data shape: {df_cleaned.shape}")
print(f"✅ Total MSAs: {len(df_cleaned)}")
print(f"\nTop 5 MSAs by total patents (2000-2015):")
print(df_cleaned.nlargest(5, 'Total')[['U.S. Regional Title', 'Total']])

Loading and cleaning patent data...
Original shape: (430, 21)
✅ Cleaned data shape: (374, 21)
✅ Total MSAs: 374

Top 5 MSAs by total patents (2000-2015):
                                 U.S. Regional Title     Total
0                 San Jose-Sunnyvale-Santa Clara, CA  143473.0
1  New York-Northern New Jersey-Long Island, NY-N...   92577.0
2                  San Francisco-Oakland-Fremont, CA   89981.0
3               Los Angeles-Long Beach-Santa Ana, CA   74381.0
4                     Boston-Cambridge-Quincy, MA-NH   62653.0


In [ ]:
"""
Load and inspect the GeoJSON file
"""
print("Loading GeoJSON data...")

# Load GeoJSON
with open('msa.geojson', 'r') as f:
    geojson_data = json.load(f)

print(f"✅ GeoJSON loaded successfully")
print(f"   Number of geographic features: {len(geojson_data['features'])}")

# Check a sample feature
sample_feature = geojson_data['features'][0]
print(f"\nSample GeoJSON properties:")
print(f"   CBSA Code: {sample_feature['properties']['cbsafp']}")
print(f"   Name: {sample_feature['properties']['name']}")

Loading GeoJSON data...
✅ GeoJSON loaded successfully
   Number of geographic features: 929

Sample GeoJSON properties:
   CBSA Code: 22380
   Name: Flagstaff, AZ


In [ ]:

"""
Merge patent data with geographic boundaries
"""
print("Merging patent data with geographic boundaries...")

# Create a mapping of CBSA codes to patent data
def merge_data_with_geojson(df, geojson, year):
    """
    Merge patent data for a specific year with GeoJSON
    """
    # Create a dictionary mapping CBSA code to patent count for the year
    patent_dict = dict(zip(df['ID Code'], df[year]))
    name_dict = dict(zip(df['ID Code'], df['U.S. Regional Title']))

    # Add patent data to GeoJSON features
    for feature in geojson['features']:
        cbsa_code = feature['properties']['cbsafp']
        feature['properties']['patent_count'] = patent_dict.get(cbsa_code, 0)
        feature['properties']['msa_name'] = name_dict.get(cbsa_code, feature['properties']['name'])

    return geojson

# Test merge with 2015 data
test_geojson = merge_data_with_geojson(df_cleaned, geojson_data.copy(), '2015')

# Count how many MSAs have patent data
msas_with_data = sum(1 for f in test_geojson['features'] if f['properties']['patent_count'] > 0)
print(f"✅ MSAs with patent data: {msas_with_data}")

Merging patent data with geographic boundaries...
✅ MSAs with patent data: 354


In [ ]:
print("Filtering GeoJSON to include only US Metropolitan Statistical Areas...")

# Create a set of valid US MSA codes from our patent data
valid_msa_codes = set(df_cleaned['ID Code'].values)

print(f"Valid US MSA codes in our dataset: {len(valid_msa_codes)}")

# Filter the GeoJSON to only include features that match our MSA codes
filtered_features = []
for feature in geojson_data['features']:
    cbsa_code = feature['properties'].get('cbsafp', '')
    if cbsa_code in valid_msa_codes:
        filtered_features.append(feature)

# Create filtered GeoJSON
geojson_data_filtered = {
    'type': 'FeatureCollection',
    'features': filtered_features
}

print(f"Original GeoJSON features: {len(geojson_data['features'])}")
print(f"Filtered GeoJSON features (US MSAs only): {len(geojson_data_filtered['features'])}")
print(f"Removed: {len(geojson_data['features']) - len(geojson_data_filtered['features'])} non-US or unmatched features")


geojson_data = geojson_data_filtered

print("\n✅ GeoJSON filtered successfully!")
print("   Now only US Metropolitan Statistical Areas will be shown")
print("   No more Mexico, Canada, or Caribbean islands!")



Filtering GeoJSON to include only US Metropolitan Statistical Areas...
Valid US MSA codes in our dataset: 374
Original GeoJSON features: 360
Filtered GeoJSON features (US MSAs only): 360
Removed: 0 non-US or unmatched features

✅ GeoJSON filtered successfully!
   Now only US Metropolitan Statistical Areas will be shown
   No more Mexico, Canada, or Caribbean islands!


In [ ]:
# ============================================================================
# CELL 8: PROPERLY BALANCED - No Cutoffs Anywhere
# ============================================================================
# Balanced margins and positioning to show everything
# ============================================================================

print("Creating properly balanced map...")

import plotly.graph_objects as go
import numpy as np
import json

# Define coordinates
msa_coordinates = {
    '41940': {'lat': 37.1, 'lon': -121.8, 'short_name': 'San Jose'},
    '35620': {'lat': 40.9, 'lon': -73.8, 'short_name': 'NYC'},
    '41860': {'lat': 38.0, 'lon': -122.4, 'short_name': 'San Francisco'},
    '31100': {'lat': 33.8, 'lon': -118.5, 'short_name': 'Los Angeles'},
    '14460': {'lat': 42.5, 'lon': -71.0, 'short_name': 'Boston'},
    '16980': {'lat': 42.1, 'lon': -87.6, 'short_name': 'Chicago'},
    '41740': {'lat': 32.4, 'lon': -117.1, 'short_name': 'San Diego'},
    '42660': {'lat': 47.8, 'lon': -122.3, 'short_name': 'Seattle'},
    '33460': {'lat': 45.2, 'lon': -93.2, 'short_name': 'Minneapolis'},
    '19820': {'lat': 42.5, 'lon': -83.3, 'short_name': 'Detroit'},
    '19100': {'lat': 32.5, 'lon': -96.5, 'short_name': 'Dallas'},
    '12420': {'lat': 29.9, 'lon': -97.7, 'short_name': 'Austin'},
}

years = ['2000', '2001', '2002', '2003', '2004', '2005', '2006',
         '2007', '2008', '2009', '2010', '2011', '2012', '2013',
         '2014', '2015']

fig = go.Figure()

# Calculate range
all_values_nonzero = []
for year in years:
    values = df_cleaned[year].dropna()
    all_values_nonzero.extend(values[values > 0].values)

z_min = min(all_values_nonzero)
z_max = max(all_values_nonzero)

# Add traces for each year
for i, year in enumerate(years):

    df_year_filtered = df_cleaned[df_cleaned[year] > 0].copy()
    year_geojson = json.loads(json.dumps(geojson_data))

    patent_dict = dict(zip(df_year_filtered['ID Code'], df_year_filtered[year]))
    name_dict = dict(zip(df_year_filtered['ID Code'], df_year_filtered['U.S. Regional Title']))

    filtered_features = []
    for feature in year_geojson['features']:
        cbsa_code = feature['properties']['cbsafp']
        if cbsa_code in patent_dict and patent_dict[cbsa_code] > 0:
            filtered_features.append(feature)

    year_geojson['features'] = filtered_features

    locations = []
    z_values = []
    hover_texts = []

    for feature in year_geojson['features']:
        cbsa_code = feature['properties']['cbsafp']
        patent_count = patent_dict[cbsa_code]
        msa_name = name_dict[cbsa_code]

        locations.append(cbsa_code)
        z_values.append(patent_count)
        hover_texts.append(f"<b>{msa_name}</b><br>Patents: {patent_count:,.0f}<br>Year: {year}")

    z_values_log = [np.log10(z + 1) for z in z_values]

    # Add choropleth
    fig.add_trace(go.Choroplethmapbox(
        geojson=year_geojson,
        locations=locations,
        z=z_values_log,
        featureidkey="properties.cbsafp",
        colorscale=[
            [0, '#f7fbff'],
            [0.2, '#deebf7'],
            [0.4, '#c6dbef'],
            [0.5, '#9ecae1'],
            [0.6, '#6baed6'],
            [0.7, '#4292c6'],
            [0.8, '#2171b5'],
            [0.9, '#08519c'],
            [1, '#08306b']
        ],
        marker_opacity=0.75,
        marker_line_width=0.5,
        marker_line_color='white',
        colorbar=dict(
            title="Patent<br>Count<br>(log scale)",
            thickness=15,
            len=0.7,
            x=1.02,
            tickmode='array',
            tickvals=[np.log10(1), np.log10(10), np.log10(100), np.log10(1000), np.log10(10000)],
            ticktext=['1', '10', '100', '1K', '10K']
        ),
        hovertemplate='%{text}<extra></extra>',
        text=hover_texts,
        visible=(i == len(years) - 1),
        showscale=(i == len(years) - 1),
        name=f'choropleth_{year}'
    ))

    # Get top 5
    top_5 = df_year_filtered.nlargest(5, year)

    lats = []
    lons = []
    texts = []
    hovers = []

    for rank, (_, row) in enumerate(top_5.iterrows(), 1):
        cbsa_code = row['ID Code']
        if cbsa_code in msa_coordinates:
            coord = msa_coordinates[cbsa_code]
            patent_count = int(row[year])

            lats.append(coord['lat'])
            lons.append(coord['lon'])
            texts.append(f"<b>{coord['short_name']}</b><br>{patent_count:,}")
            hovers.append(f"<b>#{rank}: {coord['short_name']}</b><br>Patents: {patent_count:,}")

    # Add markers
    fig.add_trace(go.Scattermapbox(
        lon=lons,
        lat=lats,
        mode='markers+text',
        marker=dict(
            size=18,
            color='#FF1744',
            opacity=1.0,
            symbol='circle'
            # The 'line' property is not supported for go.Scattermapbox markers,
            # so it has been removed to resolve the ValueError.
        ),
        text=texts,
        textposition='top center',
        textfont=dict(size=11, color='#000000', family='Arial Black'),
        hoverinfo='text',
        hovertext=hovers,
        visible=(i == len(years) - 1),
        showlegend=False,
        name=f'labels_{year}'
    ))

# Create dropdown buttons
buttons = []
for i, year in enumerate(years):
    visibility = [False] * (len(years) * 2)
    visibility[i * 2] = True
    visibility[i * 2 + 1] = True

    showscale_list = [False] * (len(years) * 2)
    showscale_list[i * 2] = True

    buttons.append(
        dict(
            label=year,
            method="update",
            args=[
                {"visible": visibility, "showscale": showscale_list},
                {"title": f"Patent Distribution Across US Metropolitan Areas - {year}<br><sub>Top 5 MSAs Labeled | Log Scale for Enhanced Visibility</sub>"}
            ]
        )
    )

# Update layout - PROPERLY BALANCED
fig.update_layout(
    title={
        'text': f"Patent Distribution Across US Metropolitan Areas - 2015<br><sub>Top 5 MSAs Labeled | Log Scale for Enhanced Visibility</sub>",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#2c3e50', 'family': 'Arial'}
    },
    mapbox=dict(
        style="carto-positron",
        zoom=3.4,
        center={"lat": 39.5, "lon": -98},
    ),
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.02,
            xanchor="left",
            y=0.97,
            yanchor="top",
            bgcolor='white',
            bordercolor='#2c3e50',
            borderwidth=2,
            font=dict(size=12),
            active=len(years) - 1
        )
    ],
    annotations=[
        dict(
            text="<b>Select Year:</b>",
            x=0.02,
            xanchor="left",
            y=1.02,
            yanchor="bottom",
            showarrow=False,
            font=dict(size=13, color='#2c3e50', family='Arial'),
            xref="paper",
            yref="paper"
        )
    ],
    height=850,
    margin={"r": 20, "t": 120, "l": 20, "b": 20}
)

print("\n✅ Properly balanced map created!")
print("   Everything should be visible now!")

fig.show()

# Summary
print("\n" + "="*70)
print(f"SUMMARY FOR 2015")
print("="*70)
top_5_2015 = df_cleaned[df_cleaned['2015'] > 0].nlargest(5, '2015')
print("\n🏆 Top 5 MSAs (marked with BRIGHT RED dots):")
for rank, (idx, row) in enumerate(top_5_2015.iterrows(), 1):
    print(f"{rank}. {row['U.S. Regional Title']}: {row['2015']:,.0f} patents")

print("\n🎉 FINAL BALANCED VERSION - Ready to publish!")

In [ ]:

# DELTA MAP: Category Filter Buttons

print("Creating delta map with category filters...")

import plotly.graph_objects as go
import numpy as np
import json
import pandas as pd

# Calculate percentage change
df_delta = df_cleaned[
    (df_cleaned['2000'] > 0) & (df_cleaned['2015'] > 0)
].copy()

df_delta['pct_change'] = ((df_delta['2015'] - df_cleaned['2000']) / df_cleaned['2000']) * 100
df_delta['abs_change'] = df_delta['2015'] - df_delta['2000']

print(f"Total MSAs: {len(df_delta)}")

# Categorize each MSA
def categorize_change(pct):
    if pct >= 150:
        return "Strong Growth"
    elif pct >= 50:
        return "Moderate Growth"
    elif pct >= -25:
        return "Stable"
    else:
        return "Decline"

df_delta['category'] = df_delta['pct_change'].apply(categorize_change)

# Print distribution
print("\n--- Category Distribution ---")
for cat in ["Strong Growth", "Moderate Growth", "Stable", "Decline"]:
    count = len(df_delta[df_delta['category'] == cat])
    pct = count / len(df_delta) * 100
    print(f"{cat}: {count} MSAs ({pct:.1f}%)")

# Create map data
delta_geojson = json.loads(json.dumps(geojson_data))

pct_change_dict = dict(zip(df_delta['ID Code'], df_delta['pct_change']))
category_dict = dict(zip(df_delta['ID Code'], df_delta['category']))
name_dict = dict(zip(df_delta['ID Code'], df_delta['U.S. Regional Title']))
val_2000_dict = dict(zip(df_delta['ID Code'], df_delta['2000']))
val_2015_dict = dict(zip(df_delta['ID Code'], df_delta['2015']))

filtered_features = []
for feature in delta_geojson['features']:
    cbsa_code = feature['properties']['cbsafp']
    if cbsa_code in pct_change_dict:
        filtered_features.append(feature)

delta_geojson['features'] = filtered_features

# Prepare data for each category
categories = ["All", "Strong Growth", "Moderate Growth", "Stable", "Decline"]

# Create figure
fig = go.Figure()

# Add a trace for each category filter
for i, filter_cat in enumerate(categories):

    locations = []
    z_values_capped = []
    hover_texts = []

    for feature in delta_geojson['features']:
        cbsa_code = feature['properties']['cbsafp']
        pct_change = pct_change_dict[cbsa_code]
        msa_category = category_dict[cbsa_code]
        msa_name = name_dict[cbsa_code]
        val_2000 = val_2000_dict[cbsa_code]
        val_2015 = val_2015_dict[cbsa_code]

        # Filter logic
        if filter_cat == "All" or msa_category == filter_cat:
            locations.append(cbsa_code)
            capped = max(-75, min(300, pct_change))
            z_values_capped.append(capped)



            hover_texts.append(
                f"<b>{msa_name}</b><br>"
                f"{msa_category}<br>"
                f"<b>Change: {pct_change:+.1f}%</b><br>"
                f"2000: {val_2000:,.0f} → 2015: {val_2015:,.0f}"
            )

    # Add choropleth trace
    fig.add_trace(go.Choroplethmapbox(
        geojson=delta_geojson,
        locations=locations,
        z=z_values_capped,
        featureidkey="properties.cbsafp",
        colorscale=[
            [0, '#d73027'],
            [0.15, '#fc8d59'],
            [0.22, '#fee090'],
            [0.25, '#ffffbf'],
            [0.35, '#d9ef8b'],
            [0.5, '#91cf60'],
            [0.7, '#1a9850'],
            [1, '#006837']
        ],
        zmin=-75,
        zmax=300,
        marker_opacity=0.85,
        marker_line_width=0.5,
        marker_line_color='white',
        colorbar=dict(
            title="% Change<br>2000→2015",
            thickness=15,
            len=0.7,
            x=1.02,
            tickmode='array',
            tickvals=[-75, -50, -25, 0, 50, 100, 150, 200, 250, 300],
            ticktext=['-75%', '-50%', '-25%', '0%', '+50%', '+100%', '+150%', '+200%', '+250%', '+300%+']
        ),
        hovertemplate='%{text}<extra></extra>',
        text=hover_texts,
        showscale=True,
        visible=(i == 0),
        name=filter_cat
    ))

# Create category filter buttons
buttons = []
for i, cat in enumerate(categories):
    # Count MSAs in this category
    if cat == "All":
        count = len(df_delta)
        label = f"All MSAs ({count})"
    else:
        count = len(df_delta[df_delta['category'] == cat])
        if cat == "Strong Growth":
            label = f"Strong Growth ({count})"
        elif cat == "Moderate Growth":
            label = f"Moderate Growth ({count})"
        elif cat == "Stable":
            label = f"Stable/Slight Change ({count})"
        else:
            label = f"Decline ({count})"

    visibility = [j == i for j in range(len(categories))]

    buttons.append(
        dict(
            label=label,
            method="update",
            args=[
                {"visible": visibility},
                {
                    "title": f"Patent Activity Change (2000-2015): {cat}<br><sub>Green  = Growth | Red  = Decline</sub>"
                }
            ]
        )
    )

# Update layout with filter buttons
fig.update_layout(
    title={
        'text': "Patent Activity Change (2000-2015): All<br><sub>Green = Growth | Red = Decline</sub>",
        'x': 0.5,
        'xanchor': 'center',
        'font': {'size': 18, 'color': '#2c3e50', 'family': 'Arial'}
    },
    mapbox=dict(
        style="carto-positron",
        zoom=3.4,
        center={"lat": 39.5, "lon": -98},
    ),
    updatemenus=[
        dict(
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            x=0.02,
            xanchor="left",
            y=0.98,
            yanchor="top",
            bgcolor='white',
            bordercolor='#2c3e50',
            borderwidth=2,
            font=dict(size=11),
            active=0
        )
    ],
    annotations=[
        dict(
            text="<b>Filter by Category:</b>",
            x=0.02,
            xanchor="left",
            y=1.04,
            yanchor="bottom",
            showarrow=False,
            font=dict(size=13, color='#2c3e50', family='Arial'),
            xref="paper",
            yref="paper"
        )
    ],
    height=850,
    margin={"r": 20, "t": 130, "l": 20, "b": 20}
)


print("\n Filter Options:")
print("   • All MSAs - see everything")
print("   • Strong Growth (150%+) - tech boom areas")
print("   • Moderate Growth (50-150%) - steady growth")
print("   • Stable (-25% to +50%) - minimal change")
print("   • Decline (<-25%) - manufacturing decline")

fig.show()

# Print insights for each category
print("\n" + "="*70)
print("CATEGORY INSIGHTS")
print("="*70)

for cat in ["Strong Growth", "Moderate Growth", "Stable", "Decline"]:
    cat_df = df_delta[df_delta['category'] == cat]
    print(f"\n{cat} ({len(cat_df)} MSAs):")
    print(f"  Range: {cat_df['pct_change'].min():.0f}% to {cat_df['pct_change'].max():.0f}%")
    print(f"  Top 3:")
    for idx, row in cat_df.nlargest(3, 'pct_change').iterrows():
        print(f"    • {row['U.S. Regional Title']}: {row['pct_change']:+.0f}%")

# Export

fig.write_html('patent_delta_filtered.html')
from google.colab import files
files.download('patent_delta_filtered.html')

